# CS-6570 Assignment #5 - Unsupervised Learning and Hypothesis Testing

**YOUR NAME HERE**

*Weber State University*

In this assignment, we'll work through some examples of unsupervised learning and hypothesis testing based on what we covered in the lectures. Please note many of these sections will require some code plus some explanation. You should use markdown for the explanations, and be sure to include both!

But first, let's import our standard libraries:

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

**PCA and K-Means**

We'll import the PCA and K-means functions from sklearn.

In [ ]:
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

Next, we'll generate a simulated data set with 20 observations in each of three classes (so 60 observations total) with 50 variables.

In [ ]:
np.random.seed(42);
X = np.random.standard_normal((60,50));

delta1 = np.random.uniform(.5,1,50)
delta2 = np.random.uniform(-1,-.5,50)

X[:20,:] += delta1
X[20:40,:] += delta2

Perform PCA on the 60 observations and plot the first two principle component score vectors. Use a different color to indicate the observations in each of the three classes.

In [ ]:
#Your Code Here

Perform $K$-means clustering of the observations with $K = 3$. How well do the clusters that you obtained in $K$-means clustering compare to the true class labels?

In [ ]:
#Your Code Here

Perform $K$-means clustering with $K$ = 2. Describe your results.

In [ ]:
#Your Code Here

Now perform $K$-means clustering with $K = 4$, and describe your results.

In [ ]:
#Your Code Here

Now perform $K$-means clustering with $K = 3$ on the first two principle component score vectors, rather than the raw data. That is, perform $K$-means clustering on the $60 \times 2$ matrix of which the first columns is the first principal component score vector, and the second column is the second principal component score vector. Comment on the results.

In [ ]:
#Your Code Here

Using the [StandardScalar](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html) estimator, perform $K$-means clustering with $K = 3$ on the data *after scaling each variable to have standard deviation one*. How do these results compare to those obtained in (b)? Explain.

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler(with_std = True, with_mean = True)

In [ ]:
#Your Code Here

**Hierarchical Clustering**

Using the [USArrests](https://r-data.pmagunia.com/dataset/r-dataset-package-datasets-usarrests) dataset, perform hierarchical clustering on the states. You can find this dataset on Canvas. Also, you'll want to import the pacakages below and use the compute_linkage function defined below (from the lecture notes).

In [ ]:
from sklearn.cluster import AgglomerativeClustering
from scipy.cluster.hierarchy import dendrogram

def compute_linkage(hclust):
    """

    Create linkage matrix used to plot a dendrogram

    Follows [sklearn example](https://scikit-learn.org/stable/auto_examples/cluster/plot_agglomerative_dendrogram.html)

    Parameters
    ----------

    hclust : `sklearn.cluster.AgglomerativeClustering`
        Fitted hierarchical clustering object.

    Returns
    -------

    linkage_matrix : np.ndarray
        Array to be passed to `dendrogram` from `scipy.cluster.hierarchy`.

    """
    
    counts = np.zeros(hclust.children_.shape[0])
    n_samples = len(hclust.labels_)
    for i, merge in enumerate(hclust.children_):
        current_count = 0
        for child_idx in merge:
            if child_idx < n_samples:
                current_count += 1  # leaf node
            else:
                current_count += counts[child_idx - n_samples]
        counts[i] = current_count

    linkage_matrix = np.column_stack([hclust.children_, hclust.distances_,
                                      counts]).astype(float)
    return linkage_matrix

First, use hierarchical clustering with complete linkage and Euclidean distance to cluster the states.

In [ ]:
#Your Code Here

Cut the dendogram at a heigh that results in three distinct clusters. Which states belong to which clusters?

In [ ]:
#Your Code Here

Hierarchically cluster the states using complete linkage and Euclidean distance, *after scaling the variables to have standard deviation one*.

In [ ]:
#Your Code Here

What effect does scaling the variables have on the hierarchical clustering obtained? In your opinion, should the variables be scaled before the inter-observation dissimilarities are computed? Provide a justification for your answer.

In [ ]:
#Your Code Here

**Hypothesis Testing**

In [ ]:
X=np.random.standard_normal((20,100))

For this part of the assignment, we've simulated (above) for $m = 100$ fund managers. These data represent each fund manager's percentage returns for each of $n = 20$ months. We wish to test the null hypothesis that each fund manager's percentage returns have population mean equal to zero. Notice that we simulated the data in such a way that each fund manager's percentage returns do have population mean zero; in other words, *all* $m$ null hypotheses are true.

Conduct a one-sample $t$-test for each fund manager, and plot a histogram of the $p$-values obtained.

In [ ]:
from scipy.stats import ttest_1samp

In [ ]:
#Your Code Here

If we control Type I error for each null hypothesis at level $\alpha = .05$, then how many null hypotheses do we reject?

In [ ]:
from statsmodels.stats.multitest import multipletests as mult_test

In [ ]:
#Your Code Here

If we control the FWER at level $.05$, then how many null hypotheses do we reject?

In [ ]:
#Your Code Here

If we control the FDR at level $.05$, then how many null hypotheses do we reject

In [ ]:
#Your Code Here

Now suppose we "cherry-pick" the 10 fund managers who perform the  best in our data. If we control the FWER for just these 10 fund managers at level $.05$, then how many null hypotheses do we reject? If we control the FDR for just these 10 fund managers at level $0.05$, then how many null hypotheses do we reject?

In [ ]:
#Your Code Here

Explain why the analysis directly above (where we "cherry-picked" the data) is misleading.